In [70]:
import sqlite3

# Step 1: Connect to the SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('StudentDB.db')

# Step 2: Create a cursor object using the cursor() method
cursor = conn.cursor()

# Step 3: Create a table named 'Students'
create_table_query = """
CREATE TABLE IF NOT EXISTS Students (
    StudentID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    PERCENTAGE REAL NOT NULL,
    Hackathons_Participated INTEGER DEFAULT 0,
    Papers_Presented INTEGER DEFAULT 0,
    Teaching_Assistance INTEGER DEFAULT 0,
    Extracurricular_Involvement INTEGER DEFAULT 0
)
"""
cursor.execute(create_table_query)

# Step 4: Commit the changes
conn.commit()

print("Table 'Students' created successfully.")

Table 'Students' created successfully.


In [ ]:
cursor.execur

In [78]:
import pandas as pd
import sqlite3

# Load the CSV file into a DataFrame
csv_file_path = 'STUDENT35.csv'  # Update this with your actual file path
df = pd.read_csv(csv_file_path)

# Connect to a SQLite database (or create it)
conn = sqlite3.connect('student_database1.db')
cursor = conn.cursor()

# Create a table based on the CSV columns (modify column types as necessary)
df.columns = [col.replace(" ", "_") for col in df.columns]  # Remove spaces in column names
df.to_sql('students', conn, if_exists='replace', index=False)

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Database created successfully!")


Database created successfully!


In [104]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('student_database1.db')

# Query the database and load the data into a DataFrame
query = "SELECT * FROM students"
df = pd.read_sql(query, conn)

# Display the data
print(df)

# Close the connection
conn.close()


    id       name    gpa  hackathons_participated  papers_presented  \
0    1      Rahul   78.0                        7                 1   
1    2      AISHA   88.0                        4                 1   
2    3      ETHAN   87.0                        5                 5   
3    4       SUMA   77.0                        9                 2   
4    5      PAWAN   72.0                        3                 4   
5    6     KALYAN   57.0                        2                 2   
6    7        RAM   81.0                        3                 0   
7    8    GOVINDH   92.2                       10                 1   
8    9     SIRISH   67.3                        0                 4   
9   10    SUMANTH   84.5                        6                 5   
10  11     SHESHI   69.0                        4                 4   
11  12    VEMANTH   92.0                        3                 0   
12  13    REVATHI   67.9                        6                 1   
13  14

In [102]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('student_database1.db')
cursor = conn.cursor()

# Execute the query to get the top 3 students by final_rank
query = "SELECT * FROM students ORDER BY final_rank DESC LIMIT 3"
cursor.execute(query)

# Fetch and print the results
results = cursor.fetchall()
for row in results:
    print(row)

# Close the connection
conn.close()


(22, 'GOVARDHAN', 98.0, 7, 5, 1, 2, 42.65)
(27, 'SEETHA', 97.7, 5, 4, 1, 2, 41.98)
(19, 'VASAVI', 100.0, 3, 2, 0, 2, 41.2)


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load the data
df = pd.read_csv('STUDENT31.csv')

# Define weights and calculate the final rank score
weights = {
    'gpa': 0.4,
    'hackathons_participated': 0.2,
    'papers_presented': 0.15,
    'teaching_assistance': 0.1,
    'extracurricular_involvement': 0.15
}

df['final_rank_score'] = (
    df['gpa'] * weights['gpa'] +
    df['hackathons_participated'] * weights['hackathons_participated'] +
    df['papers_presented'] * weights['papers_presented'] +
    df['teaching_assistance'] * weights['teaching_assistance'] +
    df['extracurricular_involvement'] * weights['extracurricular_involvement']
)

# Features and target variable
X = df[['gpa', 'hackathons_participated', 'papers_presented', 'teaching_assistance', 'extracurricular_involvement']]
y = df['final_rank_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Optionally predict for the entire dataset
df['predicted_rank_score'] = model.predict(X)

# Display top records based on final rank score
top_ranked_students = df.nlargest(3, 'final_rank_score')[['name', 'final_rank_score']]  # Assuming there is a 'name' column

print("\nTop Performing Students Based on Final Rank Score:")
print(top_ranked_students)

# Optionally display top predicted rank scores
top_predicted_students = df.nlargest(3, 'predicted_rank_score')[['name', 'predicted_rank_score']]
print("\nTop Performing Students Based on Predicted Rank Score:")
print(top_predicted_students)

Mean Absolute Error: 0.70576

Top Performing Students Based on Final Rank Score:
         name  final_rank_score
21  GOVARDHAN             41.75
18     VASAVI             41.20
26     SEETHA             41.08

Top Performing Students Based on Predicted Rank Score:
         name  predicted_rank_score
21  GOVARDHAN               41.2433
26     SEETHA               41.0087
18     VASAVI               40.9179


In [130]:
import csv
import sqlite3

def create_students_table():
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('StudentDB.db')
        cursor = conn.cursor()

        # Drop the table if it already exists (for testing purposes)
        cursor.execute("DROP TABLE IF EXISTS Students")

        # Create the Students table with a Final_Score column
        cursor.execute("""
            CREATE TABLE Students (
                StudentID INTEGER PRIMARY KEY,
                Name TEXT NOT NULL,
                Percentage REAL NOT NULL,
                Hackathons_Participated INTEGER DEFAULT 0,
                Papers_Presented INTEGER DEFAULT 0,
                Teaching_Assistance INTEGER DEFAULT 0,
                Extracurricular_Involvement INTEGER DEFAULT 0,
                Final_Score REAL
            )
        """)

        # Commit changes
        conn.commit()
        print("Students table created successfully.")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        conn.close()

def calculate_final_score(percentage, hackathons, papers, teaching_assistance, extracurricular):
    return (0.4 * percentage +
            0.2 * hackathons +
            0.15 * papers +
            0.1 * teaching_assistance +
            0.15 * extracurricular)

def import_csv_to_db(csv_file):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('StudentDB.db')
        cursor = conn.cursor()

        # Open the CSV file
        with open(csv_file, 'r') as file:
            # Read the contents of the CSV file
            csv_reader = csv.reader(file)
            next(csv_reader)  # Skip the header row

            # SQL query to insert data into the Students table
            insert_query = """
                INSERT INTO Students (StudentID, Name, Percentage,
                                      Hackathons_Participated,
                                      Papers_Presented,
                                      Teaching_Assistance,
                                      Extracurricular_Involvement,
                                      Final_Score)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """

            # Insert each row into the Students table
            for row in csv_reader:
                student_id = int(row[0])
                name = row[1]
                percentage = float(row[2])
                hackathons = int(row[3])
                papers = int(row[4])
                teaching_assistance = int(row[5])
                extracurricular = int(row[6])

                # Calculate Final Score
                final_score = calculate_final_score(percentage, hackathons, papers, teaching_assistance, extracurricular)

                # Execute insert command with calculated Final Score
                cursor.execute(insert_query, (student_id, name, percentage, hackathons, papers,
                                                teaching_assistance, extracurricular, final_score))

        # Commit changes
        conn.commit()
        print("Data imported successfully from CSV.")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    except FileNotFoundError:
        print("CSV file not found.")
    finally:
        cursor.close()
        conn.close()

# Call functions to create table and import data from CSV
create_students_table()
import_csv_to_db('student37.csv')

Students table created successfully.
Data imported successfully from CSV.
